In [1]:
import sys
sys.path.append("..")

In [2]:
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
from datetime import date
from helpers import SnowflakeHelper
import json
import os

In [3]:
snowflake_helper = SnowflakeHelper()
snowflake_config = './../helpers/snowflake_config.json'
session = snowflake_helper.create_snowpark_session(snowflake_config)

[SUCCESS] Config file loaded successfully!
[SUCCESS] Snowspark Session created successfully!


In [6]:
sqf = session.sql('SELECT * FROM SHOOTING_INCIDENTS')

In [7]:
sqf.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"_AIRBYTE_RAW_ID"                     |"_AIRBYTE_EXTRACTED_AT"    |"_AIRBYTE_META"  |"VIC_RACE"      |"OCCUR_TIME"  |"X_COORD_CD"  |"INCIDENT_KEY"  |"VIC_AGE_GROUP"  |"LATITUDE"   |"PERP_RACE"     |"Y_COORD_CD"  |"STATISTICAL_MURDER_FLAG"  |"LONGITUDE"   |"VIC_SEX"  |"BORO"         |"LON_LAT"                                      |"PERP_SEX"  |"LOCATION_DESC"            |"OCCUR_DATE"  |"PRECINCT"  |"PERP_AGE_GROUP"  |"JURISDICTION_CODE"  |
----------------------------------------------------------------------------------------------------------------